In [ ]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama faiss-cpu
!pip3 install pandas streamlit python-dotenv pypdf

In [2]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import uuid, re
import pandas as pd
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
df_gemini_answers = pd.read_csv('qa-base/qa_gemini.csv')

df_gemini_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Factual,Artigo 139 do código penal não possui parágrafos.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",Input,No código penal diz que é um a três anos ou mu...
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,Factual,Está no inciso I.
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,Não,NaN
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Factual,é de dois a cinco anos.
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,Factual,"Tanto o artigo 36 no livro do Código Penal, qu..."


In [5]:
df_deepseek_answers = pd.read_csv('qa-base/qa_deepseek.csv')

df_deepseek_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",Factual,"Em difamação, a exceção da verdade somente se ..."
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Não,NaN
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",Factual,"A pena é detenção, de seis meses a três anos, ..."
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,Factual,Art. 110 fala sobre o aumento do prazo de pres...
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",Input,"A pergunta pede as penalidades relacionadas, m..."
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não,NaN


In [6]:
model_gpt = ChatOpenAI(model="gpt-4.1-mini-2025-04-14") 
model_llama = OllamaLLM(model="llama3")

In [9]:
has_hallucination_llama_ds = []
has_hallucination_llama_gm = []

In [8]:
def answerer(answer_mod, model, answer_platform, vectorstore="", hallucination_type=""):
    hallucination_types = f"has_{hallucination_type}_hallucination_llm"
    function = globals().get(hallucination_types)

    for index, row in answer_mod.iterrows():
        question = row["Pergunta"]
        answer = row["Resposta"]
        q_id = row["ID pergunta"]

        if hallucination_type == "input":
            has_hallucination_llm = function(question, answer, model)
        elif hallucination_type == "context":
            has_hallucination_llm = function(answer, model)
        elif hallucination_type == "factual":
            has_hallucination_llm = function(answer, vectorstore, model)

        existing_entry = next((item for item in answer_platform if item["id"] == q_id), None)

        if existing_entry:
            print(f"{q_id} existe")
            existing_entry[f"{hallucination_type}_hallucination_llm"] = has_hallucination_llm
        else:
            print(f"{q_id} não existe")
            answer_platform.append({"id":q_id, "question":question, "answer":answer, 
                                    f"{hallucination_type}_hallucination_reason":has_hallucination_llm})
        
        print(has_hallucination_llm)
    
    return answer_platform

In [10]:
def txt_processing_llama(answer_platform, hallucination_type=""):
    for i in answer_platform:
        llama_content = i[f"{hallucination_type}_hallucination_reason"]

        if llama_content:
            #Pega o 0 se houver alucinação ou 1, se não houver
            has_hallu = re.match(r"^([01])[.\s]", llama_content)                
            
            if has_hallu:
                i[f"{hallucination_type}_hallucination_reason"] = re.sub(r"^[01]\.?\s*", "", llama_content.strip())
                i[f"has_{hallucination_type}_hallucination"] = has_hallu.group(1)

## Alucinações de input

In [10]:
def has_input_hallucination_llm(question, answer, model):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer})
    return result

### LLama 3

#### DeepSeek

In [ ]:
input_hallu = answerer(df_deepseek_answers, model_llama, has_hallucination_llama_ds, hallucination_type="input")
txt_processing_llama(has_hallucination_llama_ds , hallucination_type="input")

input
1 não existe
0. A resposta gerada está diretamente relacionada ao tópico da pergunta feita ("Como o código trata o crime de difamação quando há a eventual prova da verdade?"), não apresentando nenhuma informação deslocada ou irrelevante em relação à pergunta original.
input
2 não existe
0. A resposta gerada está relacionada ao tópico da pergunta e não apresenta alucinação de input, pois responde especificamente sobre a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência.
input
3 não existe
0. A resposta gerada não foge do tópico da pergunta feita, tratando especificamente sobre o crime de falsificação de documento público e descrevendo as práticas que configuram esse delito, bem como a pena prevista.
input
4 não existe
0. Não houve alucinação de input.

A resposta foi direta e relacionada ao tema da pergunta feita pelo usuário, que é sobre como o Código Penal trata o crime de dano qualificado por violên

In [ ]:
has_hallucination_llama_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'input_hallucination_reason': 'A resposta não apresenta nenhuma alucinação de input, pois está diretamente relacionada à pergunta feita sobre o tratamento do crime de difamação quando há prova da verdade. A resposta fornece informações precisas e relevantes sobre o tema, sem sair do tópico principal.',
  'has_input_hallucination': '0.'},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  

In [ ]:
df_input_veredict_llama_ds = pd.DataFrame(has_hallucination_llama_ds)
df_input_veredict_llama_ds

,id,question,answer,input_hallucination_reason,has_input_hallucination
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",A resposta não apresenta nenhuma alucinação de...,0.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Esta resposta não apresenta alucinação de inpu...,0.
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",A resposta gerada não apresenta alucinação de ...,0.
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",A resposta gerada é relevante e está diretamen...,0.
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,A resposta gerada se refere ao tema da pergunt...,0.
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não há alucinação de input porque a resposta e...,0.
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,A resposta gerada não apresenta alucinação de ...,0.
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...","Não houve alucinação de input, pois a resposta...",0.
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",A resposta não apresenta alucinação de input p...,0.
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,A resposta não foge do tópico da pergunta feit...,0.


In [ ]:
df_input_veredict_llama_ds.to_csv("llm-output/input_hallucination_deepseek_llama.csv")

#### Gemini

In [ ]:
input_hallu = answerer(df_gemini_answers, model_llama, has_hallucination_llama_gm, hallucination_type="input")

input
1 existe
0
Explicação: A resposta gerada está diretamente relacionada ao tópico da pergunta feita ("Como o código trata o crime de difamação quando há a eventual prova da verdade?"), não apresentando nenhuma informação que não esteja conectada ao assunto. Portanto, não houve alucinação de input.
input
2 existe
0. Não houve alucinação de input, pois a resposta provém do tópico da pergunta feita (pena prevista para injúria) e não apresenta conexões irrelevantes ou desviantes com o assunto original. A resposta está direcionada ao que foi perguntado e fornece informações relevantes sobre a pena prevista para injúria em contextos específicos, mantendo-se dentro do tópico da pergunta.
input
3 existe
0. Não houve alucinação de input porque a resposta gerada está diretamente relacionada ao tópico da pergunta "Quais práticas configuram o crime de falsificação de documento público?", fornecendo informações precisas e relevantes sobre as condutas que configuram esse tipo de crime, mostrando

In [ ]:
txt_processing_llama(has_hallucination_llama_gm, hallucination_type="input")

In [ ]:
has_hallucination_llama_gm

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'No crime de difamação (art. 139 do Código Penal), a prova da verdade da imputação (chamada exceptio veritatis) geralmente não é admitida. Isso porque o bem jurídico protegido é a honra objetiva da pessoa, ou seja, a reputação perante terceiros. Mesmo que o fato difamatório seja verdadeiro, a sua divulgação pode ser considerada lesiva à reputação.\r\n\r\nNo entanto, o § 3º do artigo 139 traz uma exceção: a prova da verdade é admitida se a imputação se referir a fato de que o ofendido foi processado e absolvido judicialmente. Nesse caso específico, demonstrar a verdade da acusação original pode ser relevante.',
  'input_hallucination_reason': 'A resposta gerada não foge do tópico da pergunta feita pelo usuário, que é sobre o tratamento do crime de difamação no código penal. A discussão sobre a honra objetiva e a reputação perante terceiros está relacionada ao tema inic

In [ ]:
df_input_veredict_llama_gm = pd.DataFrame(has_hallucination_llama_gm)
df_input_veredict_llama_gm

,id,question,answer,input_hallucination_reason,has_input_hallucination,input_hallucination_llm
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,A resposta gerada não foge do tópico da pergun...,0\n,0\nExplicação: A resposta gerada está diretame...
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",A resposta gerada foi relevante ao tópico da p...,0.,"0. Não houve alucinação de input, pois a respo..."
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,A resposta gerada está relacionada ao tópico d...,0.,0. Não houve alucinação de input porque a resp...
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,A resposta não foge do tópico da pergunta feit...,0.,0. A resposta gerada não apresenta alucinação ...
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Esta resposta está diretamente relacionada ao ...,0.,"0. Não há alucinação de input nesse caso, pois..."
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,A resposta gerada está relacionada ao tópico d...,0.,0. A resposta gerada não apresenta alucinação ...
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,A resposta gerada não apresenta alucinação de ...,0.,0. A resposta não apresenta alucinação de inpu...
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,A resposta gerada está diretamente relacionada...,0.,0. Não houve alucinação de input. A resposta e...
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,A resposta fornecida está relacionada ao tópic...,0\n,0.\n\nA resposta foi relevante e atendeu ao qu...
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,Não houve alucinação de input. A resposta gera...,0.,0. A resposta gerada não apresentou alucinação...


In [ ]:
df_input_veredict_llama_gm.to_csv("llm-output/input_hallucination_gemini_llama.csv")

### GPT

In [14]:
def txt_processing_gpt(answer_platform, hallucination_type=""):
    for i in answer_platform:
        gpt_content = i[f"{hallucination_type}_hallucination_reason"].content

        if gpt_content:
            has_input_hallucination = re.match(r"^([01])[.\s]", gpt_content)
            i[f"{hallucination_type}_hallucination_reason"] = re.sub(r"content='.*.'|^[01]\.?\s*", "", gpt_content.strip())

            i[f"has_{hallucination_type}_hallucination"] = has_input_hallucination[0]

In [13]:
has_hallucination_gpt_ds = []
has_hallucination_gpt_gm = []

#### DeepSeek

In [ ]:
input_hallu = answerer(df_deepseek_answers, model_gpt, has_hallucination_gpt_ds, hallucination_type="input")

input
1 não existe
content='0 A resposta está diretamente relacionada à pergunta sobre como o código trata o crime de difamação em caso de prova da verdade, explicando claramente as condições previstas no artigo 139 do Código Penal e exceções conforme o artigo 140, §1º. Portanto, não houve alucinação de input.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 365, 'total_tokens': 427, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BdRLg2nLZMRVbL3i4ERkDyiyFiCxG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3572e4ae-ccdb-4b69-adfa-2a675c5a9f53-0' usage_metadata={'input_tokens': 365, 'output_tokens': 62, 'total_tokens': 427, 'input_token_det

In [ ]:
txt_processing_gpt(has_hallucination_gpt_ds, hallucination_type="input")

In [ ]:
has_hallucination_gpt_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'input_hallucination_reason': 'A resposta está diretamente relacionada à pergunta sobre como o código trata o crime de difamação em caso de prova da verdade, explicando claramente as condições previstas no artigo 139 do Código Penal e exceções conforme o artigo 140, §1º. Portanto, não houve alucinação de input.',
  'has_input_hallucination': '0 '},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com defici

In [ ]:
df_input_veredict_gpt_ds = pd.DataFrame(has_hallucination_gpt_ds)
df_input_veredict_gpt_ds

,id,question,answer,input_hallucination_reason,has_input_hallucination
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",A resposta está diretamente relacionada à perg...,0
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",A resposta está estritamente relacionada à per...,0
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...","Não houve alucinação de input, pois a resposta...",0.
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",Não houve alucinação de input porque a respost...,0.
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não houve alucinação de input porque a respost...,0.
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...","Não houve alucinação de input, pois a resposta...",0.
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,"Não houve alucinação de input, pois a resposta...",0.
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",A resposta está estritamente relacionada ao tó...,0
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",Não houve alucinação de input porque a respost...,0.
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não houve alucinação de input porque a respost...,0.


In [ ]:
df_input_veredict_gpt_ds.to_csv("llm-output/input_hallucination_deepseek_gpt.csv")

#### Gemini

In [ ]:
input_hallu = answerer(df_gemini_answers, model_gpt, has_hallucination_gpt_gm, hallucination_type= "input")
txt_processing_gpt(has_hallucination_gpt_gm, hallucination_type="input")

input
1 não existe
content='0 Houve a resposta está diretamente relacionada à pergunta sobre como o código trata o crime de difamação quando há prova da verdade. A explicação aborda corretamente o tratamento legal da prova da verdade no contexto do artigo 139 do Código Penal, sem fugir do tópico. Portanto, não há alucinação de input.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 410, 'total_tokens': 474, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BdRNa8BeS3HR1ZLHejDC0FUXEWxgt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--54327ab7-215c-484d-b8b9-430675c38b3d-0' usage_metadata={'input_tokens': 410, 'output_tokens': 64, 'total_tokens

In [ ]:
has_hallucination_gpt_gm

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'No crime de difamação (art. 139 do Código Penal), a prova da verdade da imputação (chamada exceptio veritatis) geralmente não é admitida. Isso porque o bem jurídico protegido é a honra objetiva da pessoa, ou seja, a reputação perante terceiros. Mesmo que o fato difamatório seja verdadeiro, a sua divulgação pode ser considerada lesiva à reputação.\r\n\r\nNo entanto, o § 3º do artigo 139 traz uma exceção: a prova da verdade é admitida se a imputação se referir a fato de que o ofendido foi processado e absolvido judicialmente. Nesse caso específico, demonstrar a verdade da acusação original pode ser relevante.',
  'input_hallucination_reason': 'Houve a resposta está diretamente relacionada à pergunta sobre como o código trata o crime de difamação quando há prova da verdade. A explicação aborda corretamente o tratamento legal da prova da verdade no contexto do artigo 139

In [ ]:
df_input_veredict_gpt_gm = pd.DataFrame(has_hallucination_gpt_gm)
df_input_veredict_gpt_gm

,id,question,answer,input_hallucination_reason,has_input_hallucination
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Houve a resposta está diretamente relacionada ...,0
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...","Não houve alucinação de input, pois a resposta...",0.
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,"Não houve alucinação de input, pois a resposta...",0.
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,A resposta está claramente relacionada ao crim...,0
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...","Não houve alucinação de input, pois a resposta...",0.
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Não houve alucinação de input porque a respost...,0.
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,A resposta está diretamente relacionada ao tem...,0
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Houve resposta direta e pertinente à pergunta ...,0
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não houve alucinação de input porque a respost...,0.
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,A resposta está diretamente relacionada à perg...,0.


In [ ]:
df_input_veredict_gpt_gm.to_csv("llm-output/input_hallucination_gemini_gpt.csv")

## Alucinações contextuais

In [11]:
#Alucinação contextual
def has_context_hallucination_llm(answer, model):
    contextHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Texto: Foram observadas "primeiras propagações" do vírus na população de países situados fora do continente americano, 
            assinalou o director-geral adjunto da OMS, citando o Reino Unido, o Japão e a Austrália, além do Chile, na América do Sul.

        Explicação: Chile não está fora do continente americano.
        
        Exemplo 2: 
            Texto: Ele só compra leite de vaca, pois é intolerante à lactose.

        Explicação: Se ele é intolerante a lactore, não pode beber leite de vaca.

        Exemplo 3:
            Texto: O gato miava muito alto. A temperatura hoje é de 25 graus
        
        Explicação: Os assuntos não se conectam entre si.

        Caso haja alucinação contextual, responda com 1. Caso não tenha alucinação contextual, responda com 0.
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que o texto seja "{answer}", houve alucinação contextual?
    """
    prompt = ChatPromptTemplate.from_template(contextHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"answer": answer })

    return result

### Llama 3

#### Deepseek

In [16]:
llama_context_hallucination_dps = answerer(df_deepseek_answers, model_llama, 
                                         has_hallucination_llama_ds, hallucination_type="context")

1 não existe
0. Não há alucinação contextual.

Explicação: O texto se refere a um tema específico sobre crime e direito, e a explicação apresentada é lógica e consistente com o contexto. Nenhuma contradição ou incoerência foi encontrada entre as informações apresentadas.
2 não existe
0. A justificativa para não haver alucinação contextual é que o texto descreve leis e penalidades relacionadas à injúria, sem apresentar contradições entre as informações.
3 não existe
0. Não houve alucinação contextual.

A explicação é que o texto apresenta apenas uma descrição legal, sem qualquer conexão com informações previas ou posteriores, nem contexto que possa ser considerado contraditório. O texto apresenta informações sobre um crime e sua pena, não há alucinação contextual.
4 não existe
0.

A resposta não apresenta contradição ou falta de conexão entre os assuntos. O texto trata sobre o crime de dano e suas consequências legais, sem apresentar qualquer tipo de alucinação contextual.
5 não existe


In [18]:
txt_processing_llama(has_hallucination_llama_ds, hallucination_type="context")

In [19]:
has_hallucination_llama_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'context_hallucination_reason': 'Não há alucinação contextual.\n\nExplicação: O texto se refere a um tema específico sobre crime e direito, e a explicação apresentada é lógica e consistente com o contexto. Nenhuma contradição ou incoerência foi encontrada entre as informações apresentadas.',
  'has_context_hallucination': '0'},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  'answer': 

In [20]:
df_context_veredict_llama_ds = pd.DataFrame(has_hallucination_llama_ds)
df_context_veredict_llama_ds

,id,question,answer,context_hallucination_reason,has_context_hallucination
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",Não há alucinação contextual.\n\nExplicação: O...,0
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",A justificativa para não haver alucinação cont...,0
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",Não houve alucinação contextual.\n\nA explicaç...,0
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",A resposta não apresenta contradição ou falta ...,0
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Explicação: O texto apresenta uma descrição do...,0
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não há alucinação contextual.\n\nExplicação: O...,0
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,Não houve alucinação contextual.\n\nA justific...,0
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",Não houve alucinação contextual.\n\nA explicaç...,0
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",Não há alucinação contextual.\n\nA resposta nã...,0
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não houve alucinação contextual.\n\nA razão pe...,0


In [21]:
df_context_veredict_llama_ds.to_csv("llm-output/context_hallucination_deepseek_llama.csv")

#### Gemini

In [ ]:
llama_context_hallucination_gm = answerer(df_gemini_answers, model_llama, 
                                           has_hallucination_llama_gm, hallucination_type="context")

In [23]:
txt_processing_llama(has_hallucination_llama_ds, hallucination_type="context")

In [26]:
df_context_veredict_llama_gm = pd.DataFrame(has_hallucination_llama_gm)
df_context_veredict_llama_gm

,id,question,answer,context_hallucination_reason
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,0. Não há alucinação contextual nesse texto. O...
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",0. Não houve alucinação contextual.\n\nA respo...
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,0. Não houve alucinação contextual.\n\nA expli...
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,0. Não há alucinação contextual nesse texto po...
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",0. Não há alucinação contextual nesse texto. O...
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,0. Não há alucinação contextual nesse texto.\n...
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,0.\n\nExplicação: O texto apresenta uma seqüên...
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,0. Não há alucinação contextual. O texto está ...
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,0.\n\nExplicação: O texto apresenta uma lista ...
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,0. Não há alucinação de contexto nesse texto. ...


In [27]:
df_context_veredict_llama_gm.to_csv("llm-output/context_hallucination_gemini_llama.csv")

### GPT

#### DeepSeek

In [ ]:
gpt_context_hallucination_dps = answerer(df_deepseek_answers, model_gpt, 
                                         has_hallucination_gpt_ds, hallucination_type="context")

In [ ]:
txt_processing_gpt(has_hallucination_gpt_ds, hallucination_type="context")

In [ ]:
df_factual_veredict_gpt_ds = pd.DataFrame(has_hallucination_gpt_ds)
df_factual_veredict_gpt_ds

In [ ]:
df_factual_veredict_gpt_ds.to_csv("llm-output/context_hallucination_deepseek_gpt.csv")

#### Gemini

In [ ]:
gpt_context_hallucination_gm = answerer(df_gemini_answers, model_gpt, 
                                         has_hallucination_gpt_gm, hallucination_type="context")

In [ ]:
df_factual_veredict_gpt_gm = pd.DataFrame(has_hallucination_gpt_gm)
df_factual_veredict_gpt_gm

In [ ]:
df_factual_veredict_gpt_gm.to_csv("llm-output/context_hallucination_gemini_gpt.csv")

## Alucinações factuais

In [11]:
pdf_loader = PyPDFLoader("pdf/codigo_penal.pdf")
load_codigo_penal = pdf_loader.load()

In [12]:
codigo_penal = ""

for page in load_codigo_penal:
    codigo_penal += page.page_content

In [13]:
def has_factual_hallucination_llm(answer, vectorstore, model):
    FactualHallucinationTemplate = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto:
        {context}

        Considerando a resposta "{answer}", houve alucinação factual?
"""
    prompt_template = ChatPromptTemplate.from_template(FactualHallucinationTemplate)
    
    query = """Pergunta: {question}\nResposta: {answer}"""
    retrieve = vectorstore.similarity_search(query)

    for retrieved in retrieve:
        context = "\n".join(str(retrieved))

    chain = prompt_template | model

    result = chain.invoke({"context":context, "answer": answer })

    return result

In [14]:
def vectorstore(embedding):
    txt_splitter = RecursiveCharacterTextSplitter(chunk_size=1400, 
                                                    chunk_overlap=120, 
                                                    length_function=len,
                                                    separators=["\n\n", "\n", " "])

    chunks = txt_splitter.split_documents(load_codigo_penal)

    vectorstore = FAISS.from_documents(chunks, 
                                       embedding=embedding)
    
    return vectorstore

### Llama 3

In [ ]:
if ("has_hallucination_llama_ds" in locals() or
    "has_hallucination_llama_gm" in locals()):
    pass
else:
    has_hallucination_llama_ds = []
    has_hallucination_llama_gm = []

hallucination_type = "factual"

In [ ]:
embedding_llama = OllamaEmbeddings(model='nomic-embed-text')

In [ ]:
vectorstore_llama = vectorstore(embedding_llama)

#### DeepSeek

In [ ]:
llama_factual_hallucination_dps = answerer(df_deepseek_answers, model_llama, has_hallucination_llama_ds, 
                                           vectorstore=vectorstore_llama, hallucination_type="factual")

factual


In [ ]:
has_hallucination_llama_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'input_hallucination_reason': 'A resposta gerada está diretamente relacionada ao tema da pergunta e não foge do tópico da difamação e seu tratamento no código penal brasileiro.',
  'has_input_hallucination': '0.'},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  'answer': 'A injúria (art. 140, CP) é ofender a dignidade de alguém.\r\n\r\nQualificadora: \r\nSe a injúria utilizar elemento

In [ ]:
txt_processing_llama(has_hallucination_llama_ds, hallucination_type="factual")

KeyError: 'factual_hallucination_reason'

In [ ]:
has_hallucination_llama_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'factual_hallucination_reason': 'A resposta parece ser uma descrição correta do artigo 139 e 140 do Código Penal Brasileiro (CP) sobre o crime de difamação, incluindo a situação em que há prova da verdade. Não há informações falsas na resposta, portanto não há alucinação factual.',
  'has_factual_hallucination': '0.'},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  'answer': 'A injúri

In [ ]:
df_factual_veredict_llama_ds = pd.DataFrame(has_hallucination_llama_ds)
df_factual_veredict_llama_ds

,id,question,answer,factual_hallucination_reason,has_factual_hallucination
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",A resposta parece ser uma descrição correta do...,0.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",A resposta fornecida é uma descrição correta d...,0\n
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",A resposta fornecida parece ser factual e corr...,0\n
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",A resposta fornecida é uma reprodução exata do...,0.
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,A resposta fornecida é uma descrição precisa e...,0.
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",A razão é que a resposta apresentada está corr...,0.
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,A resposta fornecida é uma descrição correta d...,0.
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",A resposta fornecida é uma informação factual ...,0.
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",A resposta fornecida é uma descrição correta d...,0.
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,"A resposta fornecida é coerente e fidedigna, n...",0.


In [ ]:
df_factual_veredict_llama_ds.to_csv("llm-output/factual_hallucination_deepseek_llama.csv")

#### Gemini

In [ ]:
llama_factual_hallucination_dps = answerer(df_gemini_answers, model_llama, has_hallucination_llama_gm, 
                                           vectorstore=vectorstore_llama, hallucination_type="factual")

TypeError: answerer() got multiple values for argument 'vectorstore'

In [ ]:
txt_processing_llama(has_hallucination_llama_gm, hallucination_type)

In [ ]:
df_factual_veredict_llama_gm = pd.DataFrame(has_hallucination_llama_gm)
df_factual_veredict_llama_gm

,id,question,answer,factual_hallucination_reason,has_factual_hallucination
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,A resposta fornece uma descrição precisa e cor...,0.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",Explicação: A resposta fornecida é correta e b...,0.
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,Explicação: A resposta fornecida não apresenta...,0.
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,A resposta parece correta e factualmente basea...,0.
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Não há alucinação factual.\n\nA resposta forne...,0.
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,A resposta fornecida é uma informação factual ...,0.
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,A resposta apresentada parece ser factual e co...,0.
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,"A resposta fornecida é factual e correta, pois...",0.
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Explicação: A resposta apresentada está corret...,0\n
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,Explicação: A resposta fornecida é factual e n...,0.


In [ ]:
df_factual_veredict_llama_gm.to_csv("llm-output/factual_hallucination_gemini_llama.csv")

### GPT 

In [17]:
has_hallucination_gpt_ds = []
has_hallucination_gpt_gm = []

In [18]:
embedding_gpt = OpenAIEmbeddings(model="text-embedding-ada-002")

In [19]:
vectorstore_gpt = vectorstore(embedding_gpt)

#### DeepSeek

In [20]:
gpt_factual_hallucination_dps = answerer(df_deepseek_answers, model_gpt, 
                                           has_hallucination_gpt_ds, vectorstore=vectorstore_gpt, hallucination_type="factual")

factual
1 não existe
content='0. Não houve alucinação factual, pois a resposta está correta ao definir o crime de difamação conforme o artigo 139 do Código Penal brasileiro e apresenta corretamente as exceções relacionadas à prova da veracidade do fato, em conformidade com o artigo 140, §1º, do mesmo código.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 1011, 'total_tokens': 1074, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-Ber3gPUyny4JBAtFrt3ygCRi3LMTH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c1398ca6-904f-4c2d-907c-a47a6f0993d6-0' usage_metadata={'input_tokens': 1011, 'output_tokens': 63, 'total_tokens': 1074, 'input_token_d

In [21]:
txt_processing_gpt(has_hallucination_gpt_ds, hallucination_type="factual")

In [22]:
has_hallucination_gpt_ds

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'factual_hallucination_reason': 'Não houve alucinação factual, pois a resposta está correta ao definir o crime de difamação conforme o artigo 139 do Código Penal brasileiro e apresenta corretamente as exceções relacionadas à prova da veracidade do fato, em conformidade com o artigo 140, §1º, do mesmo código.',
  'has_factual_hallucination': '0.'},
 {'id': 2,
  'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiê

In [23]:
df_factual_veredict_gpt_ds = pd.DataFrame(has_hallucination_gpt_ds)
df_factual_veredict_gpt_ds

,id,question,answer,factual_hallucination_reason,has_factual_hallucination
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...","Não houve alucinação factual, pois a resposta ...",0.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Houve alucinação factual porque a pena qualifi...,1.
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",Houve alucinação factual porque o trecho apres...,1
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",A resposta apresenta alucinação factual porque...,1
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não houve alucinação factual na resposta. O ar...,0.
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Houve alucinação factual porque a resposta ind...,1.
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,A resposta está correta ao afirmar que o prazo...,0
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",Houve alucinação factual na resposta porque o ...,1.
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...","Não há alucinação factual na resposta, pois os...",0.
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,"Não há alucinação factual na resposta, pois os...",0.


In [24]:
df_factual_veredict_gpt_ds.to_csv("llm-output/factual_hallucination_deepseek_gpt.csv")

#### Gemini

In [25]:
gpt_factual_hallucination_dps = answerer(df_gemini_answers, model_gpt, 
                                           has_hallucination_gpt_gm, vectorstore=vectorstore_gpt, hallucination_type="factual")

factual
1 não existe
content='0. Não houve alucinação factual porque a resposta está de acordo com o que prevê o artigo 139 do Código Penal brasileiro sobre difamação. De fato, a prova da verdade da imputação (exceptio veritatis) não é geralmente admitida para proteger a honra objetiva. A exceção mencionada no § 3º do artigo 139, que permite a prova da verdade quando o ofendido foi processado e absolvido, está correta conforme o texto legal.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 1056, 'total_tokens': 1153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-Ber6G3WmhP0EA7RhWB7nFYrkXcfl7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run-

In [26]:
txt_processing_gpt(has_hallucination_gpt_gm, hallucination_type="factual")

In [27]:
has_hallucination_gpt_gm

[{'id': 1,
  'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'No crime de difamação (art. 139 do Código Penal), a prova da verdade da imputação (chamada exceptio veritatis) geralmente não é admitida. Isso porque o bem jurídico protegido é a honra objetiva da pessoa, ou seja, a reputação perante terceiros. Mesmo que o fato difamatório seja verdadeiro, a sua divulgação pode ser considerada lesiva à reputação.\r\n\r\nNo entanto, o § 3º do artigo 139 traz uma exceção: a prova da verdade é admitida se a imputação se referir a fato de que o ofendido foi processado e absolvido judicialmente. Nesse caso específico, demonstrar a verdade da acusação original pode ser relevante.',
  'factual_hallucination_reason': 'Não houve alucinação factual porque a resposta está de acordo com o que prevê o artigo 139 do Código Penal brasileiro sobre difamação. De fato, a prova da verdade da imputação (exceptio veritatis) não é geralmente admitida para

In [28]:
df_factual_veredict_gpt_gm = pd.DataFrame(has_hallucination_gpt_gm)
df_factual_veredict_gpt_gm

,id,question,answer,factual_hallucination_reason,has_factual_hallucination
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Não houve alucinação factual porque a resposta...,0.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...","Não houve alucinação factual, pois a descrição...",0.
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,"A resposta está correta, pois o artigo 297 do ...",0.
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,A resposta está correta em afirmar que o crime...,0.
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",A resposta está correta ao descrever o crime d...,0.
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Houve alucinação factual porque o artigo menci...,1.
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,"Houve alucinação factual na resposta, pois o C...",1.
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,"Não houve alucinação factual, pois a resposta ...",0.
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não houve alucinação factual porque a resposta...,0.
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,A resposta está correta e não apresenta alucin...,0


In [29]:
df_factual_veredict_gpt_ds.to_csv("llm-output/factual_hallucination_gemini_gpt.csv")